In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from sklearn.utils import class_weight
from sklearn.preprocessing import OneHotEncoder
import csv

# Read data for X and y
X = pd.read_csv("X.csv").values
y = pd.read_csv("y.csv").values

# Flatten list y
y = [item for sublist in y for item in sublist]

In [7]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from imblearn.over_sampling import SMOTE

# Apply SMOTE to balance the dataset
#sm = SMOTE(random_state=42)
#X_res, y_res = sm.fit_resample(X, y)

oversampler = RandomOverSampler(random_state=42)
X_res, y_res = oversampler.fit_resample(X,y)

In [8]:
# Determine the number of unique critical error types
num_classes = len(np.unique(y_res))
class_names = np.unique(y_res)

# Encode the target variable using LabelEncoder and one-hot encoding
label_encoder = LabelEncoder()
y_res = label_encoder.fit_transform(y_res)
y_res = to_categorical(y_res)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [9]:
from keras.models import Model
from keras.layers import Input, Conv1D, BatchNormalization, Activation, MaxPooling1D, Add, GlobalAveragePooling1D, Dense

def residual_block(input_tensor, filters, kernel_size, strides=1):
    x = Conv1D(filters, kernel_size, strides=strides, padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv1D(filters, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    
    # Downsample the input tensor if strides=2
    if strides == 2:
        input_tensor = Conv1D(filters, 1, strides=strides, padding='same')(input_tensor)
    
    x = Add()([x, input_tensor])
    x = Activation('relu')(x)
    return x

# Input layer
inputs = Input(shape=(300, 1))

# Initial convolution
x = Conv1D(64, 7, padding='same')(inputs)
x = BatchNormalization()(x)
x = Activation('relu')(x)

# Residual blocks
x = residual_block(x, 64, 3)
x = residual_block(x, 64, 3)
x = residual_block(x, 64, 3)

x = residual_block(x, 128, 3, strides=2)
x = residual_block(x, 128, 3)
x = residual_block(x, 128, 3)

x = residual_block(x, 256, 3, strides=2)
x = residual_block(x, 256, 3)
x = residual_block(x, 256, 3)

# Global average pooling
x = GlobalAveragePooling1D()(x)

# Output layer
outputs = Dense(12, activation='softmax')(x)

# Create the model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300, 1)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 300, 64)      512         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 300, 64)     256         ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 300, 64)      0           ['batch_normalization[0][0]']

                                                                                                  
 batch_normalization_9 (BatchNo  (None, 150, 128)    512         ['conv1d_10[0][0]']              
 rmalization)                                                                                     
                                                                                                  
 activation_9 (Activation)      (None, 150, 128)     0           ['batch_normalization_9[0][0]']  
                                                                                                  
 conv1d_11 (Conv1D)             (None, 150, 128)     49280       ['activation_9[0][0]']           
                                                                                                  
 batch_normalization_10 (BatchN  (None, 150, 128)    512         ['conv1d_11[0][0]']              
 ormalization)                                                                                    
          

                                                                                                  
 activation_18 (Activation)     (None, 75, 256)      0           ['add_8[0][0]']                  
                                                                                                  
 global_average_pooling1d (Glob  (None, 256)         0           ['activation_18[0][0]']          
 alAveragePooling1D)                                                                              
                                                                                                  
 dense (Dense)                  (None, 12)           3084        ['global_average_pooling1d[0][0]'
                                                                 ]                                
                                                                                                  
Total params: 1,484,044
Trainable params: 1,478,540
Non-trainable params: 5,504
_____________________________

In [10]:
# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50
 75/394 [====>.........................] - ETA: 1:37 - loss: 2.1603 - accuracy: 0.2775

KeyboardInterrupt: 

In [ ]:
import numpy as np
y_pred = model.predict(X_test)

y_pred_classes = []
y_real_classes = []

for item in y_pred:
    y_pred_classes.append(np.argmax(item))

for item in y_test:
    y_real_classes.append(np.argmax(item))
    
print(y_pred_classes[0])
print(y_real_classes[0])
# Assuming your classes are labeled from 0 to 25
predicted_class = np.argmax(y_pred[0])
probability = y_pred[0][predicted_class]

print("Predicted class:", predicted_class)
print("Probability:", probability)

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

print(classification_report(y_real_classes, y_pred_classes))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

cm = confusion_matrix(y_real_classes, y_pred_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()